In [1]:
import numpy as np
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# Words embeddings

En el siguiente trabajo utilizaremos word embeddings. Recordemos que los word embeddings son representaciones vectoriales de las palabras ($\mathbb{R}^d$, generalmente $25 \leq d \leq 1000$).

En el presente trabajo corroboraremos propiedades de los word embeddings y lo utilizaremos para clasificar texto.

## Gensim
Gensim es un paquete muy usado, eficiente y escalable para trabajar con similitud de texto.

Existen en internet diferentes base de datos de "word embeddings" que se pueden descargar: generalmente archivos ".vec" que asoscian cada palabra con un vector de enteros. Estos archivos pueden ser cargados por gensim.

Como vimos en la clase, los embeddings son calculados por medio de un proceso de entrenamiento auto-supervisado. Existen varios algoritmos para producir word embeddings. El objetivo es siempre, utilizar dichos embeddings luego para otras tareas como clasificación de texto.

### Standford GloVe

Para palabras en inglés utilizaremos los [GloVe word vectors](https://nlp.stanford.edu/projects/glove/) que pueden ser descargados de [aquí](https://nlp.stanford.edu/data/glove.6B.zip).

### Cargando los modelos

Hemos guardado nuestros embeddings en la subcarpeta ('./models/') de nuestro repositorio. Esta estructura es muy común es proyectos de Machine Learning. Los word embeddings son considerados modelos ya que son representacioens aprendidas de los datos que "modelan" relaciones semánticas de palabras.


In [4]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

embeddings_path = '../models/glove.6B/glove.6B.300d.txt'

glove_file = datapath(embeddings_path)
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")

glove2word2vec(embeddings_path, word2vec_glove_file)

model_en = KeyedVectors.load_word2vec_format(word2vec_glove_file)

C:\Users\hgarnica\AppData\Local\Temp\ipykernel_16980\1349107516.py:9: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(embeddings_path, word2vec_glove_file)


Podemos corroborar que efectivamente a cada palabra le asigna un vector (de dimensión 300 en este caso):

In [5]:
model_en['obama'].shape

(300,)

In [6]:
model_en['obama'][:10]

array([ 0.10303 ,  0.48005 , -0.50917 ,  0.142   , -0.58965 ,  0.17514 ,
        0.084485, -0.56529 ,  0.10185 , -1.76    ], dtype=float32)

Gensim nos provee funcionalidad para hacer operaciones útiles con estos vectores como buscar otros vectores similares de forma eficiente.

Si bien todavía no vimos una utilidad práctica, que el resultado "tenga sentido" nos habla de que las representaciones realmente tienen potencial de ser útiles: las palabras "relacionadas" están cerca.

In [7]:
model_en.most_similar('obama')

[('barack', 0.9254721999168396),
 ('mccain', 0.7590768337249756),
 ('bush', 0.7570987939834595),
 ('clinton', 0.7085603475570679),
 ('hillary', 0.6497915387153625),
 ('kerry', 0.6144053339958191),
 ('rodham', 0.613863468170166),
 ('biden', 0.5940852165222168),
 ('gore', 0.5885975956916809),
 ('democrats', 0.5608304142951965)]

A .most_similar también le podemos pasar un vector. En este caso, como no sabe de que palabra proviene el vector que le estamos pasando, la misma palabra "obama" aparecerá en el top ya que tiene similaridad 1 (en el otro caso gensim quita del top el resultado trivial de la misma palabra):

In [8]:
model_en.most_similar(model_en['obama'])

[('obama', 1.0),
 ('barack', 0.9254721999168396),
 ('mccain', 0.7590768337249756),
 ('bush', 0.7570987939834595),
 ('clinton', 0.7085603475570679),
 ('hillary', 0.6497915387153625),
 ('kerry', 0.6144053339958191),
 ('rodham', 0.6138635277748108),
 ('biden', 0.5940852165222168),
 ('gore', 0.5885975956916809)]

Es importante tener en cuenta que si bien los embeddings codifican información, esa información tiene que estar disponible en el dataset en el que fueron entrenados. En este caso, en el momento en el que los embeddings fueron entrenados, trump no estaba asosciado con la presidencia de los estados unidos, y los embeddings similares al embedding de "trump" tienen que ver con los negocios y familia de Donald Trump:

In [9]:
model_en.most_similar('trump')

[('ivana', 0.4999052882194519),
 ('melania', 0.45651501417160034),
 ('casino', 0.45222753286361694),
 ('nows', 0.44631344079971313),
 ('knauss', 0.4360748529434204),
 ('hilton', 0.4234515130519867),
 ('trumps', 0.41433754563331604),
 ('ivanka', 0.40609344840049744),
 ('resorts', 0.3992827832698822),
 ('wynn', 0.3902420997619629)]

Otra propiedad de los word embeddings que nos habla de su potencialidad de capturar significado son las "meaning components": si pensamos al embedding $e_w$ de la palabra $w$ como un punto en el espacio $\mathbb{R}^{300}$ que representa la palabra $w$, luego el vector diferencia $e_b - e_a$ entre los embeddings de dos palabras $a$ y $b$, representarán la dirección de "a hacia b". Esta dirección puede tener un significado preciso si se piensa por ejemplo $e_{australian} - e_{australia}$ es la dirección de la palabra "australia" hacia "australian", o sea es la dirección que indica como transformar un país en una nacionalidad. 

![title](meaning_component.webp)

Se puede corroborar que sumando esa misa dirección a otro país como "mexican", se obtiene un punto muy cercano a la respectiva nacionalidad "mexican"

In [10]:
from_country_to_nationality = model_en['australian'] - model_en['australia']
model_en.most_similar(from_country_to_nationality + model_en['mexico'])

[('mexican', 0.8576371669769287),
 ('mexico', 0.7306691408157349),
 ('colombian', 0.5733058452606201),
 ('venezuelan', 0.5602331757545471),
 ('argentine', 0.5432359576225281),
 ('spanish', 0.5182245373725891),
 ('bolivian', 0.5015351176261902),
 ('peruvian', 0.5009693503379822),
 ('chilean', 0.4896966516971588),
 ('tijuana', 0.46829572319984436)]

Gensim provee de funcionalidad para hacer esto en una sola linea, con los argumentos positive y negative de la función most_similar. Además, excluye de las búsqueda a las palabras de "positive" (notar que "mexico" no aparece ya entre los resultados) ya que son resultados triviales.

In [11]:
model_en.most_similar(positive=['australian', 'mexico'], negative=['australia'])

[('mexican', 0.8613249659538269),
 ('colombian', 0.5716720819473267),
 ('venezuelan', 0.5595687031745911),
 ('argentine', 0.5421301126480103),
 ('spanish', 0.5189327001571655),
 ('bolivian', 0.5003634095191956),
 ('peruvian', 0.5000406503677368),
 ('chilean', 0.48860830068588257),
 ('tijuana', 0.47381317615509033),
 ('american', 0.4659227132797241)]

En otras palabras, cuando hacemos esto estamos calculando analogías: "$X_1$ es a $X_2$ como $Y_1$ es a ...".

Podemos implementar una función para calcular analogías:

In [13]:
def analogy(model, x1, x2, y1):
    result = model.most_similar(positive=[y1, x2], negative=[x1])
    return result[0][0]

Las analogías capturan significados:
- De género.
- Gramaticales.
- Información factual ("Es capital de ...")

In [14]:
analogy(model_en, 'man', 'king', 'woman')

'queen'

In [15]:
analogy(model_en, 'walk', 'walked', 'think')

'thought'

In [16]:
analogy(model_en, 'germany', 'berlin', 'ukraine')

'kiev'

#### Ethical warning

Es importante recalcar que los modelos, así como capturan información factual de los datos de entrenamiento, también capturan sesgos y pueden repetirlos.

En este caso le hicimos calcular la siguiente analogía: "{man, he, gentleman} es a programmer como {women, she, lady} es a ..."
Y el modelo contesta entre sus respuestas probables: actress, pregnant, mother.

In [14]:
model_en.most_similar(positive=['woman', 'she', 'lady', 'programmer'], negative=['man', 'he', 'gentleman'])

[('actress', 0.4465254247188568),
 ('herself', 0.4092317521572113),
 ('her', 0.4046404957771301),
 ('pregnant', 0.40257561206817627),
 ('mother', 0.3984251618385315),
 ('linda', 0.39298215508461),
 ('sister', 0.3816814720630646),
 ('michelle', 0.3761267364025116),
 ('mary', 0.3757179081439972),
 ('laura', 0.3712882995605469)]

#### Más funcionalidad de Gensim

Gensim contiene muchas funciones extras como .doesnt_match, detectar las palabra que no pertenece a cierto grupo de palabras.
Podemos ver todas las funciones disponibles de nuestros word vectors [aquí](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [17]:
print(model_en.doesnt_match("breakfast cereal dinner lunch".split()))

cereal


#### Ejercicio 1: "Embeddings en español" (obligatorio)

Para palabras en español utilizaremos embeddings entrenados con FastText que hemos descargado del [repositorio](https://github.com/dccuchile/spanish-word-embeddings). Particularmente el archivo utilizado se puede descargar de [aquí](https://zenodo.org/record/3234051/files/embeddings-l-model.vec).

El ejercicio consiste en:
1. Cargar los embeddings linkeados utilizados la librería Gensim
2. Dar 1 ejemplo de most_similar para una palabra en español.
3. Dar 1 ejemplo de un "fallo" similar a lo que ocurría con "trump" en most_similar para una palabra en español: donde el embedding desconozca cierta información por el año en dodne fue entrenado.
4. Dar 5 ejemplos de analogías. Es necesario reescribir la función analogy?
5. Dar 1 ejemplo donde el modelo replique sesgos presentes en los datos que considere pernicioso.



#### Ejercicio 1 (solución)

Tratar de resolver sin mirar la solución.


In [18]:
model_es = KeyedVectors.load_word2vec_format('../models/embeddings-l-model.vec')

In [19]:
model_es.most_similar('asado')

[('asada', 0.7902636528015137),
 ('asados', 0.7901856899261475),
 ('guiso', 0.7752070426940918),
 ('estofado', 0.7731484174728394),
 ('asadas', 0.770538330078125),
 ('pollo', 0.761725664138794),
 ('chorizo', 0.7508561015129089),
 ('asador', 0.7376899123191833),
 ('churrasco', 0.7269676923751831),
 ('guisado', 0.7253041863441467)]

In [22]:
model_es.most_similar('messi')

[('neymar', 0.772021472454071),
 ('ronaldinho', 0.7494037747383118),
 ('tévez', 0.7455589771270752),
 ('higuaín', 0.7395713329315186),
 ('maradona', 0.7223870754241943),
 ('zinedine', 0.7179876565933228),
 ('forlán', 0.7102377414703369),
 ('falcao', 0.7088690996170044),
 ('kaká', 0.7069801092147827),
 ('ibrahimović', 0.7039478421211243)]

In [23]:
model_es.most_similar('milei')

[('mileidy', 0.6162314414978027),
 ('archilei', 0.5704389810562134),
 ('milessi', 0.564062237739563),
 ('milea', 0.5424215793609619),
 ('zilei', 0.5419692993164062),
 ('leilei', 0.5339961051940918),
 ('montironi', 0.5273367762565613),
 ('jucilei', 0.5159411430358887),
 ('iannaccone', 0.5134119987487793),
 ('miledi', 0.512912392616272)]

In [26]:
analogy(model_es, 'hombre', 'cocinero', 'mujer')

'cocinera'

In [27]:
analogy(model_es, 'habalar', 'hablando', 'comer')

'comiendo'

In [28]:
analogy(model_es, 'alemania', 'berlin', 'francia')

'paris'

In [29]:
analogy(model_es, 'argentina', 'maradona', 'brasil')

'ronaldinho'

In [30]:
analogy(model_es, 'harry', 'rowling', 'anillos')

'tolkien'

In [31]:
women_terms_es = ['mujer', 'ella', 'chica']
men_terms_es = ['hombre', 'el', 'chico']
model_es.most_similar(positive=women_terms_es+['obrero'], negative=men_terms_es)

[('trabajadora', 0.5435466766357422),
 ('costurera', 0.48017582297325134),
 ('lesbiana', 0.4611666202545166),
 ('obradora', 0.4599410593509674),
 ('obrera', 0.4563603699207306),
 ('vividora', 0.44954487681388855),
 ('tipógrafa', 0.44897523522377014),
 ('cizkova', 0.4488752484321594),
 ('pañera', 0.4467610716819763),
 ('socialadora', 0.4435802400112152)]

In [32]:
model_es.most_similar(positive=men_terms_es+['obrera'], negative=women_terms_es)

[('obrero', 0.41873669624328613),
 ('reaccionario', 0.4009731113910675),
 ('revolucionario', 0.39405152201652527),
 ('améwicano', 0.39144861698150635),
 ('proletario', 0.39072349667549133),
 ('maderoso', 0.3807927668094635),
 ('burgués', 0.3793873190879822),
 ('sindicalismo', 0.3717118799686432),
 ('copador', 0.3708193004131317),
 ('aunario', 0.3698670566082001)]

## Utilizando Word Embedding para clasificar texto

Éstas propiedades deseables analizadas nos dicen que los embeddings calculados tienen sentido, no son simplemente vectores aleatorios. Pero ... sirven para algo práctico?

Resulta que como, no es de sorprender, son buenas features para utilizar en algoritmos de Machine Learning. Por ejemplo si queremos clasificar texto podemos crear features del texto utilizando los word embeddings.

Mostraremos este hecho en el dataset [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/) que contiene reviews de películas y el respectivo "sentimiento": "positivo" o "negativo", según si el usuario acompañó su review con un voto positivo o negativo.

Nosotros ya lo hemos descargado en la carpeta "../data/aclImb/". El dataset tiene una estructura bastante típica de los datasets (no solo de texto):
- Contiene dos subcarpetas "train" y "test": el dataset ya define cuál será el conjunto de test para que los resultados de diferentes trabajos sean comparables. Si vamos a sacar datos para validación (porque queremos probar varias configuraciones de hiperparámetros), tendremos que utilizar los datos de la carpeta "train" (en este sentido estos datos serían los denominados datos de "desarrollo", o sea todos los datos sobre los cuales vamos a tomar decisiones, tanto validación como training).
- Dentro de cada carpeta "train" y "test" existen subcarpetas "pos" y "neg" que contienen archivos de texto. Cada archivo es una review. La clase de cada review se la da el nombre de la subcarpeta donde se encuentra ("pos" y "neg").
- Existen otros archivos que vamos a ignorar para esta tarea. Muchas veces los datasets incluyen otra información como metadatos o incluso datos para otros tipos de tareas.

In [1]:
import os.path 
import glob
import pandas as pd

dataset_path = '../data/aclImdb/'

cls = ['pos', 'neg']

def get_text(file_path):
    with open(file_path, 'r') as f:
        return '\n'.join(f.readlines())

def load_dataset(is_train, limit):
    data_path = os.path.join(dataset_path, 'train' if is_train else 'test')
    data = []
    limit_per_class = limit//2
    for c in cls:
        class_path = os.path.join(data_path,c)
        regex_glob = os.path.join(class_path,"*.txt")
        for i, file_path in enumerate(glob.glob(regex_glob)):
            if i == limit_per_class:
                break
            data.append((get_text(file_path),c))
    return pd.DataFrame(data=data,columns=['text', 'class'])
            
        
df = load_dataset(is_train=True, limit=100000).sample(frac=1)
df.head(10)

,text,class


Podemos visualizar el primer comentario que es claramente negativo ("This movie is terrible"). Visualizar también nos es útil para entender la dificultad de la tarea. Como podemos ver las reviews son largas, llenas de información dificilmente relevante para el objetivo de la tarea:

In [28]:
print(df.iloc[0]['text'])

I saw this movie when i was much younger and i thought it was funny. I saw it again last week, and you can guess the result. Some funny parts in it, very few and too long. The beginning is the only thing that is funny if you ask me.<br /><br />If you want a total b-movie this is a good pick, but don't expect too much from aliens dwarf size


Nuestras features (o variable independiente) son textos. Cómo obtener un array de features del texto dado nuestros word embeddings?

Primero que nada, es necesario calcular la secuencia de palabras que aparece en el texto.

Para calcular las features del texto existen diferentes formas. Siempre implicará calcular los embeddings de las palabras para luego:
1. Calcular la suma.
2. Calcular la media
3. Calcular otro agregado (max, min, etc). O una combinación de ellos.
4. Concatenarlos. En este caso hay que decidir cuántos embeddings vamos a concatenar y agregar padding (embedding de ceros) para rellenar comentarios cortos y truncar comentarios largos. Recordar que todos los samples tienen que tener la misma cantidad de features para utilizar una algoritmos de Machine Learning.

Nosotros vamos a optar por el punto 2

In [29]:
import nltk

text = 'This movie is terrible. I watch crappy movies for fun.'
words = nltk.word_tokenize(text.lower())
words

['this',
 'movie',
 'is',
 'terrible',
 '.',
 'i',
 'watch',
 'crappy',
 'movies',
 'for',
 'fun',
 '.']

In [30]:
def get_text_embedding(text):
    words = nltk.word_tokenize(text.lower())
    l = [model_en[w] for w in words if w in model_en]
    if not l:
        return np.zeros((300,))
    return np.array(l).mean(axis=0)
get_text_embedding(text).shape 

(300,)

Llego el momento de calcular los embeddings de todo el texto:

In [31]:
import tqdm

def get_df_embeddings(df):
    embs = []
    for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
        embs.append(get_text_embedding(row['text']))
    embs=np.array(embs)
    return embs

In [32]:
X = get_df_embeddings(df)

100%|███████████████████| 25000/25000 [00:20<00:00, 1236.83it/s]


Y calcularemos las targets (variable dependiente):

In [33]:
y = (df['class']=='pos').astype(int)
y

22148    0
5888     1
24595    0
16694    0
2011     1
        ..
10607    1
14079    0
1754     1
13910    0
15415    0
Name: class, Length: 25000, dtype: int64

Nos separaremos un split de validación:

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((20000, 300), (5000, 300), (20000,), (5000,))

Y finalmente entrenaremos y evaluaremos algunos clasificadores simples: uno basado en KNN y otro basado en RandomForest.

In [35]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=11)
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=11)

In [38]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

RandomForestClassifier()

In [39]:
pred_rf = rf_clf.predict(X_valid)
pred_knn = knn_clf.predict(X_valid)

probs_rf = rf_clf.predict_proba(X_valid)
probs_knn = knn_clf.predict_proba(X_valid)
ensamble_pred = ((probs_rf[:,1]+probs_knn[:,1])/2.0)>0.5

pred_rf.shape, pred_knn.shape, ensamble_pred.shape

((5000,), (5000,), (5000,))

In [40]:
((probs_rf[:,1]>0.5) == pred_rf).all()

True

In [41]:
((probs_knn[:,1]>0.5) == pred_knn).all()

True

In [42]:
from sklearn.metrics import accuracy_score

print(f'Random Forest acc: {accuracy_score(y_valid,pred_rf)}')
print(f'KNN acc: {accuracy_score(y_valid,pred_knn)}')
print(f'Ensamble acc: {accuracy_score(y_valid,ensamble_pred)}')

Random Forest acc: 0.7774
KNN acc: 0.7454
Ensamble acc: 0.7742


#### Ejercicio 2: "Otros text embedding" (opcional)

Probar otro método de generar el text embedding a partir de los word embeddings y reportar los resultados.

#### Ejercicio 3: "Conjunto de test" (obligatorio)

Reportar los resultados en el conjunto de test.

Entrenar los resultados en toda la data de desarrollo (incluyendo x_valid), y volver a evaluar el test. Esto no se debería hacer, por qué?

En la práctica, no obstante es común entrenar la configuración elegida en toda la data de desarrollo y luego evaluar en el test. Notar que esto no supone un conflicto siempre y cuando evaluemos en test UNA SOLA VEZ (no vale elegir entre el modelo entrenado en toda la data de desarrollo.
Que ventajas tiene hacer esto?

#### Ejercicio 4: "MeLi Challenge 2019" (opcional)

El ejercicio consiste en entrenar un clasificador para el dataset del melichallenge2019 y evaluarlo utilizando la métrica que especificaba la competencia ([Balanced Accuracy Binary Classification](https://neptune.ai/blog/balanced-accuracy)). Utilizar el dataset disponible en el link de Kaggle provisto a continuación. Para entrenar utilizar las primeras 18,000,000 filas y reservar las últimas 2,000,000 de filas. Como el dataset es público no podrá controlar que no hagan trampa, pero confío en la honestidad intelectual de cada uno: sólo pueden evaluar el split de test una única vez, al final.

#### MeLi Challenge 2019

En 2019 Mercado Libre publicó en su Challenge anual un problema de Machine Learning en donde había que predecir la categoría de productos dado el título (y el idioma que era español y portugués).

El premio era una entrada (con viaje y alojamiento) a KHIPU: un evento latinoamericano MUY importante de inteligencia artificial que se año se celebró en noviembre.

El dataset público (que te daban para entrenar y validar) está [subido a Kaggle](https://www.kaggle.com/datasets/abugim/meli-data-challenge-2019). Lamentablemente no publicaron el dataset de test, oculto a momento de competir, que utilizaron para el rankeo final de las soluciones, así que no podemos compararnos exactamente contra el scoreboard final.

Algunas soluciones encontradas en internet:
- Usando BERT y NNML: 0.8789  ([link](https://github.com/eduardofv/meli2019)).
- Usando LSTM se obtuvo una puntación de: 0.8853 ([link](https://github.com/Sebasu11/MERCADOLIBRE-DATA-CHALLENGE-2019)).
- Contando n-gramas de palabras: 0.89207 ([link](https://github.com/isadofschi/MeLiDataChallenge)).
- **(Ganador de la competencia)** Usando embeddings FastText + LSTM y GRU + clasificador de capas densas: 0.91733 ([link](https://github.com/eduagarcia/meli-challenge-2019)).

Como se puede observar no siempre utilizar modelos más complejos (como BERT en ese entonces era de total vanguardia) nos garantiza mejores resultados que métodos tradicionales (como LSTM) o incluso que métodos muy simples como n-gramas.

In [43]:
df=pd.read_csv('../data/meli2019/meli2019.csv')
df

,title,label_quality,language,category
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES
...,...,...,...,...
19999995,Brochas De Maquillaje Kylie Set De 12 Unidades,unreliable,spanish,MAKEUP_BRUSHES
19999996,Trimmer Detailer Wahl + Kit Tijeras Stylecut,reliable,spanish,HAIR_CLIPPERS
19999997,Bateria Portátil 3300 Mah Power Bank Usb Max...,unreliable,portuguese,PORTABLE_CELLPHONE_CHARGERS
19999998,"Palo De Hockey Grays Nano 7 37,5''",unreliable,spanish,FIELD_HOCKEY_STICKS


El método describe nos provee info útil de cada columna, como la cantidad de valores únicos:

In [ ]:
df.describe()

In [ ]:
len(df)*0.1